In [1]:
import torch
import torchvision
from torch.autograd import Variable

In [2]:
try:
    import baseline_model
    import self_defined_dataset
except:
    !jupyter nbconvert --to script baseline_model.ipynb
    !jupyter nbconvert --to script self_defined_dataset.ipynb
finally:
    import baseline_model
    import self_defined_dataset

In [3]:
if torch.cuda.is_available():
    CUDA_USAGE = True
else:
    CUDA_USAGE = False

In [4]:
transform_train_list = [
    torchvision.transforms.Resize((256, 256), interpolation=3),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]
data_transform = torchvision.transforms.Compose(transform_train_list)

In [5]:
transform_val_list = [
    torchvision.transforms.Resize((256, 256), interpolation=3),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]

In [6]:
img_root_path = "/home/extension/kaggle/APTOS_2019_Blindness_Detection/train_images"
img_csv = "/home/extension/kaggle/APTOS_2019_Blindness_Detection/train.csv"

In [7]:
train_dataset = self_defined_dataset.Blindness(img_root_path, img_csv, True, data_transform)
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)

In [8]:
model = baseline_model.Blindness(4)
if CUDA_USAGE:
    model = model.cuda()
else:
    pass

In [9]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, [40, 60, 80])

In [10]:
EPOCH = 100

In [11]:
for tmp_epoch in range(EPOCH):
    for data in dataloader:
        img_data, label = data
        
        print(img_data.size())
        if CUDA_USAGE:
            input_data = Variable(img_data.cuda())
            label = Variable(label.cuda())
        else:
            pass
        
        optimizer.zero_grad()
        
        outputs = model(input_data)
        loss = loss_function(outputs, label)
        
        print(loss)
        loss.backward()
        optimizer.step()
        
    scheduler.step()

torch.Size([8, 3, 256, 256])


RuntimeError: cuda runtime error (59) : device-side assert triggered at /pytorch/aten/src/THC/THCReduceAll.cuh:327